# Look for demographics, co-occurring terms in blog descriptions that mention fandoms

In [1]:
# Load blog descriptions
import pandas as pd

fandom_blog_descriptions_path = '/data/tumblr_community_identity/fandom_blog_descriptions_processed.csv'
blog_info = pd.read_csv(fandom_blog_descriptions_path, sep='\t')
fandoms = blog_info['fandom'].unique()
print(len(fandoms))
blog_info

/usr0/home/mamille2/anaconda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


33


,tumblelog_id,activity_time_epoch,tumblr_blog_name,tumblr_blog_title,tumblr_blog_description,tumblr_blog_url,is_group_blog,is_primary,is_private,created_time_epoch,...,timezone,language,blog_classifier,generated_date,tumblog_id,post_count,fandom,fandom_term,processed_tumblr_blog_description,tokenized_blog_description
0,123758,1538018782,coldfashion,AP ROCKS YOUR FACE,annelise pruitt i designed the emmy-winning st...,http://coldfashion.tumblr.com/,False,False,False,NaN,...,NaN,NaN,safe,20181021,123758,25,star_wars,star wars,annelise pruitt i designed the emmy-winning st...,annelise pruitt i designed the emmy-winning st...
1,139093,1558154235,hauntedfalcon,Haunted Jaeger,fuck Reylo,https://hauntedfalcon.tumblr.com/,False,False,False,1.196747e+12,...,US/Eastern,en_US,safe,20190518,139093,982,star_wars,reylo,fuck Reylo,fuck reylo
2,340785,1546861232,anomalies,party.,"<p>27 years old <br>i like dogs, pokemon, taki...",http://anomalies.tumblr.com/,False,False,False,1.216068e+12,...,US/Pacific,en_US,safe,20190201,340785,2,pokemon,pokemon,"27 years old i like dogs, pokemon, taking pict...","27 years old i like dogs , pokemon , taking pi..."
3,493522,1556894408,hazal,Smitten hedgehogs like tea,"<br>Hazal. <br> 29 years old, vegetarian, web...",https://hazal.tumblr.com/,False,False,False,1.225645e+12,...,Europe/Istanbul,en_US,safe,20190619,493522,10,supernatural,spn,"Hazal. 29 years old, vegetarian, web designer,...","hazal . 29 years old , vegetarian , web design..."
4,794605,1563544355,sundayriver81,Sundayriver's world,"Italian girl, obsessed with The walking Dead a...",https://sundayriver81.tumblr.com/,False,False,False,1.236731e+12,...,Europe/Amsterdam,en_US,safe,20190719,794605,999,walking_dead,the walking dead,"Italian girl, obsessed with The walking Dead a...","italian girl , obsessed with the walking dead ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319111,444485579,1541257544,bhluetina,lilyy,<p>potterhead.</p><p>the marauders’</p>,https://bhluetina.tumblr.com/,False,False,False,1.541255e+12,...,US/Eastern,en_US,safe,20181201,444485579,4,harrypotter,potterhead,potterhead. the marauders’,potterhead . the marauders ’
319112,444669851,1541965412,zzellak,Zellak,30/ He/Him/ Them/They. 18+ art. NO MINORS OR ...,https://zzellak.tumblr.com/,False,False,False,1.541347e+12,...,US/Eastern,en_US,adult,20181114,444669851,17,undertale,undertale,30/ He/Him/ Them/They. 18+ art. NO MINORS OR ...,30/ he/him/ them/they . 18+ art . no minors or...
319113,444881744,1567866289,peaches-and-nicotine,♡,"ED TW! || 18 || UK || Not Pro || 5'6"" sw: 218l...",https://peaches-and-nicotine.tumblr.com/,False,False,False,1.541449e+12,...,US/Eastern,en_US,safe,20190908,444881744,88,teen_wolf,tw,"ED TW! || 18 || UK || Not Pro || 5'6"" sw: 218l...",ed tw ! || 18 || uk || not pro || 5 ' 6 '' sw ...
319114,446026777,1542862673,littlegleekhunter,Pretty Gleek Hunter❤,Pretty Little Liars/Supernatural & Glee❤😍,https://littlegleekhunter.tumblr.com/,False,False,False,1.542070e+12,...,US/Eastern,en_US,safe,20190616,446026777,4,supernatural,supernatural,Pretty Little Liars/Supernatural & Glee❤😍,pretty little liars/supernatural & glee❤😍


In [2]:
fandoms = [
'academia',
'allmarvel',
'arthurian',
'attack_on_titan',
'dcu',
'dr_who',
'dragonage',
'haikyuu',
'hamilton',
'harrypotter',
'homestuck',
'korra',
'naruto',
'once_upon_a_time',
'overwatch',
'pokemon',
'power_rangers',
'riverdale',
'sherlock',
'song_ice_fire',
'star_trek',
'star_wars',
'stargate',
'supergirl',
'supernatural',
'teen_wolf',
'tolkien',
'undertale',
'voltron',
'walking_dead',
]

In [ ]:
# Search for ages
import re
from tqdm import tqdm_notebook as tqdm

pattern = r'^\d+$'
# patterns = [ # overkill
#  r'(?:[^-+\w]|^)([1-5]{1}[0-9]{1})[^-+0-9]|^([1-5]{1}[0-9]{1})$',
# #   r'twelve',
#   r'thirteen',
#   r'fourteen',
#   r'fifteen',
#   r'sixteen',
#   r'seventeen',
#   r'eighteen',
#   r'nineteen',
#    r'(twenty|thirty|forty|fifty)([ -](one|two|three|four|five|six|seven|eight|nine))?',
#    r'\bxvi{0-3}\b',
#    r'\bxix\b',
#    r'\bxxi{0-3}\b',
#    r'\bxxiv\b',
#    r'\bxxvi{0-3}\b',
#    r'\bxxix\b',
#  ]

excl =  ['nsfw 18', '18 nsfw', '18 only', 'only 18', '18\+', '18 \+', '18 or older', 'at least 18', 'under 18', '18×', '18 plus',
                 '24 hours',
                 r'(\d{1,2}|\d{4})[\/\.-](\d|[0-3][0-9])[\/\.-](\d{2}|\d{4})(\W|$)',
                 r'(^|\D)\d{1,2}[\/\.-](\d{2}|\d{4})(\W|$)',
                 r'\d* ?(jan\w*|feb\w*|mar\w*|apr\w*|may|jun|june|jul|july|aug\w*|sep\w*|oct\w*|nov\w*|dec\w*)[ ,.]\d*[ \W,.]*\d*',
                 r'\d+[:.]',
                 r'u?(c|g|s)w[1-3]?: ?\d*(kg|lb)?',
                 r'\d* ?(kg|lb|kilograms|pounds|days?)',
                 r'24[\/\* -]7',
                 r'\$\d*|\d*\$',
                 r'\d*(st|nd|rd|th)',
                 '30 rock',
                 '15 lat',
        ]

def remove_age_exclusions(text):
    new_text = text
    for p in excl:
        new_text = re.sub(p, ' ', new_text)
    return new_text
         
outlines = []
demographic_counts = {}
# removed_ages = [str(n) for n in range(81,100)]
# age_normalizer = {str(n): n for n in range(13,81)}
# age_normalizer.update({
#     'thirteen': 13,
#     'fourteen': 14,
#   'sixteen': 16,
#   'seventeen': 17,
#   'eighteen':18,
#   'nineteen':19,
#     'twenty': 20,
#     'twenty-one': 21,
#     'twenty-two': 22,
#     'twenty-three': 23,
#     'twenty-four': 24,
#     'twenty-five': 25,
#   'xv': 15,
#   'xix': 29,
#   'xxi': 21,
#   'xxiv': 24,
#   'xxv': 25,
#   'xxix': 29,
# })
# for fandom in fandoms:
#     demographic_counts[fandom] = {}
#     print(fandom)
#     fandom_blog_descriptions = blog_info.loc[blog_info['fandom']==fandom, 'processed_tumblr_blog_description']
         
#     # Exclude terms
#     fandom_blog_descriptions = map(remove_age_exclusions, fandom_blog_descriptions)
         
#     # Search blog descriptions
#     for pattern in patterns:
#         matches += [re.search(pattern, desc).group() for desc in fandom_blog_descriptions if re.search(pattern, desc) is not None]
#         ages = [m for m in matches if not m in removed_ages]
#         ages_problem = [a for a in ages if not a in age_normalizer]
        
        
#     outlines.append([fandom, *[demographic_counts[fandom][group_name] for group_name in grouped_patterns], len(fandom_blog_descriptions)])
# pd.DataFrame(outlines, columns=['fandom'] + list(grouped_patterns.keys()) + ['total_descriptions']).set_index('fandom')

from collections import Counter
print('Removing age exclusions...')
blog_info['tokenized_blog_description'] = [remove_age_exclusions(desc) for desc in tqdm(blog_info['tokenized_blog_description'].tolist())]
fandom_terms = blog_info.groupby('fandom').agg({'tokenized_blog_description': lambda x: ' '.join(x.tolist())})
fandom_words = fandom_terms['tokenized_blog_description'].to_dict()
fandom_word_counters = {}
for fandom, words in fandom_words.items():
    fandom_word_counters[fandom] = Counter([w for w in words.split() if not w in stops])

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)

age_df = pd.DataFrame(index=range(15,81))
# age_df.index = range(15,81)
for fandom, ctr in fandom_word_counters.items():
    print(fandom)
    if fandom in ['ajin', 'world_ends_with_you', 'youjo senki']:
        continue
#     matches = [item for item in ctr.most_common() if re.search(pattern, item[0]) is not None and \
#             int(item[0]) < 85 and int(item[0]) > 14 and \
#             item[1] >= 5]
#     for word, count in matches:
#         print(f'{word}\t{count}')
    matches = {word: ctr[word] for word in ctr if re.search(pattern, word) is not None and \
            int(word) <= 80 and int(word) >= 15}
    age_df[fandom] = [matches.get(str(i), 0) for i in range(15,81)]
#     for i in range(15,81):
# #         print(f'{i}\t{matches.get(str(i), 0)}')
#         print(matches.get(str(i), 0))
#     print()
age_df

In [15]:
# Search for gendered terms
import re
import pdb

# Non-exclusive groupings of interest
grouped_patterns = {'female': [
                r'female\b',
                r'^(?!.*(super|wonder|bat)).*\bgirl\b',
                r'^(?!.*(super|wonder|bat)).*\bwoman\b', r'gu+rl', r'\bgii+rl',
                r'\bmom\b', r'mum',
                r'wife', 
                r'\bsister\b',
#                 r'\bprincess\b', r'queen', # led to miscounts for Once
                r'\blady\b', r'daughter', r'mommy',
            ],
            'male': [
                r'\bmale\b',
                r'boy\b',
                r'^(?!.*(super|wonder|bat)).*man\b',
                r'guy\b', 
                r'\bdad\b', 
                r'husband', 
                r'\bbrother\b', r'\bson\b', 
            ],
            'nonbinary': [
                r'non-binary', r'nonbinary', r'\bnb\b', r'agender', r'neutrois', r'androgynous',
                r'bigender', 
                r'genderfluid', r'gender-fluid',
            ],
            'trans': [
                r'(?!.*transformers)trans', r'ftm', r'mtf', 
            ], 
#             'straight': [ # really cis/non-aligned as an estimate of non-lgbtq, but bad
#                 r'\bcis',
#                 r'^(?!.*(super|wonder|bat|trans)).*\bmale\b',
#                 r'^(?!.*(super|wonder|bat|trans)).*\bfemale\b',
#                 r'^(?!.*(super|wonder|bat|trans)).*\bman\b',
#                 r'^(?!.*(super|wonder|bat|trans)).*woman\b',
#                 r'^(?!.*(super|wonder|bat|trans)).*\bgirl\b',
#                 r'^(?!.*(super|wonder|bat|trans)).*boy\b',
#             ],
            'lgbtq': [
                r'non-binary', r'nonbinary', r'\bnb\b', r'agender', r'neutrois', r'androgynous',
                r'bigender', 
                r'genderfluid', r'gender-fluid',
                r'^(?!.*transformers)trans', r'ftm', r'mtf', 
                r'lgbt',
                r'queer',
            ]
           }

outlines = []
demographic_counts = {}
for fandom in fandoms:
    demographic_counts[fandom] = {}
    print(fandom)
    fandom_blog_descriptions = blog_info.loc[blog_info['fandom']==fandom, 'tokenized_blog_description']
    for group_name, patterns in grouped_patterns.items():
        matches = []
        for desc in fandom_blog_descriptions:
            for pattern in patterns:
                if re.search(pattern, desc) is not None:
                    matches.append(desc)
                    break
        group_count = len(matches)
#         if fandom == 'once_upon_a_time' and group_name == 'female':
#             pdb.set_trace()
        demographic_counts[fandom][group_name] = group_count
    outlines.append([fandom, *[demographic_counts[fandom][group_name] for group_name in grouped_patterns], len(fandom_blog_descriptions)])
#         print(f'{group_name}\t{group_count}')
#         print()
pd.DataFrame(outlines, columns=['fandom'] + list(grouped_patterns.keys()) + ['total_descriptions']).set_index('fandom')

academia
allmarvel
arthurian
attack_on_titan
dcu
dr_who
dragonage
haikyuu
hamilton
harrypotter
homestuck
korra
naruto
once_upon_a_time
overwatch
pokemon
power_rangers
riverdale
sherlock
song_ice_fire
star_trek
star_wars
stargate
supergirl
supernatural
teen_wolf
tolkien
undertale
voltron
walking_dead


,female,male,nonbinary,trans,lgbtq,total_descriptions
fandom,,,,,,
academia,404,536,120,110,240,8624
allmarvel,2540,2796,255,323,943,37002
arthurian,249,232,14,19,80,2937
attack_on_titan,276,379,20,41,49,3733
dcu,438,556,46,111,157,8493
dr_who,169,149,18,30,52,1834
dragonage,540,380,99,90,251,6384
haikyuu,232,298,29,36,61,4306
hamilton,350,318,77,76,197,5116


In [16]:
# Search for sexual orientation
import re
import pdb
from tqdm import tqdm_notebook as tqdm

# Non-exclusive groupings of interest
grouped_patterns = {
    'gay': [
        r'gay',
        r'lesbian', r'\bhomo',
        r'\bwlw\b', r'\bmlm\b',
            ],
    'bi/pan': [
        r'bisexual', r'\bbi\b', r'pansexual', r'\bpan\b', 
    ],
    'straight': [
        r'straight',
        r'\bcishet',
    ],
    'aro/ace': [
        r'\bace\b', r'\basexual', r'aro-ace', r'aro/ace',
    ],
    'lgbtq': [
        r'gay',
        r'lesbian', r'\bhomo',
        r'\bwlw\b', r'\bmlm\b',
        r'bisexual', r'\bbi\b', r'pansexual', r'\bpan\b', 
        r'\bace\b', r'\basexual', r'aro-ace', r'aro/ace',
        r'lgbt',
        r'queer',
        r'demisexual',
    ]
}

outlines = []
demographic_counts = {}
for fandom in fandoms:
    demographic_counts[fandom] = {}
    print(fandom)
    fandom_blog_descriptions = blog_info.loc[blog_info['fandom']==fandom, 'tokenized_blog_description']
    for group_name, patterns in tqdm(grouped_patterns.items()):
        matches = []
        for desc in fandom_blog_descriptions:
            for pattern in patterns:
                if re.search(pattern, desc) is not None:
                    matches.append(desc)
                    break
#         for pattern in patterns:
#             matches += [desc for desc in fandom_blog_descriptions if re.search(pattern, desc) is not None]
        group_count = len(matches)
#         if fandom == 'voltron' and group_name == 'bi/pan':
#             pdb.set_trace()
        demographic_counts[fandom][group_name] = group_count
    outlines.append([fandom, *[demographic_counts[fandom][group_name] for group_name in grouped_patterns], len(fandom_blog_descriptions)])
#         print(f'{group_name}\t{group_count}')
#         print()
pd.DataFrame(outlines, columns=['fandom'] + list(grouped_patterns.keys()) + ['total_descriptions']).set_index('fandom')

academia



allmarvel



arthurian



attack_on_titan



dcu


...


tolkien



undertale



voltron



walking_dead


,gay,bi/pan,straight,aro/ace,lgbtq,total_descriptions
fandom,,,,,,
academia,270,409,26,245,952,8624
allmarvel,945,1788,120,503,3652,37002
arthurian,57,91,9,48,240,2937
attack_on_titan,104,88,17,68,274,3733
dcu,270,263,58,65,644,8493
dr_who,61,55,10,31,167,1834
dragonage,217,352,19,179,829,6384
haikyuu,133,100,8,181,416,4306
hamilton,196,354,21,110,712,5116


In [17]:
# Search for pronouns
import re

grouped_patterns = {
    'male': [
        r'(?:\W|\b)he(?:\W|\b)', r'(?:\W|\b)him(?:\W|\b)',
    ],
    'female': [
        r'(?:\W|\b)she(?:\W|\b)', r'(?:\W|\b)her(?:\W|\b)',
    ],
    'nonbinary': [
        r'(?:\W|\b)they(?:\W|\b)', r'(?:\W|\b)them(?:\W|\b)',
        r'(?:\W|\b)xe(?:\W|\b)', r'(?:\W|\b)xem(?:\W|\b)',
        r'it/its',
        r'theythem',
    ]
}

outlines = []
demographic_counts = {}
for fandom in fandoms:
    demographic_counts[fandom] = {}
    print(fandom)
    fandom_blog_descriptions = blog_info.loc[blog_info['fandom']==fandom, 'tokenized_blog_description']
    for group_name, patterns in tqdm(grouped_patterns.items()):
        matches = []
        for desc in fandom_blog_descriptions:
            for pattern in patterns:
                if re.search(pattern, desc) is not None:
                    matches.append(desc)
                    break
        group_count = len(matches)
#         if fandom == 'voltron' and group_name == 'bi/pan':
#             pdb.set_trace()
        demographic_counts[fandom][group_name] = group_count
    outlines.append([fandom, *[demographic_counts[fandom][group_name] for group_name in grouped_patterns], len(fandom_blog_descriptions)])
#         print(f'{group_name}\t{group_count}')
#         print()
pd.DataFrame(outlines, columns=['fandom'] + list(grouped_patterns.keys()) + ['total_descriptions']).set_index('fandom')

academia
allmarvel
arthurian
attack_on_titan
dcu
dr_who
dragonage
haikyuu
hamilton
harrypotter
homestuck
korra
naruto
once_upon_a_time
overwatch
pokemon
power_rangers
riverdale
sherlock
song_ice_fire
star_trek
star_wars
stargate
supergirl
supernatural
teen_wolf
tolkien
undertale
voltron
walking_dead


,male,female,nonbinary,total_descriptions
fandom,,,,
academia,317,1114,628,8624
allmarvel,889,3019,1612,37002
arthurian,77,192,184,2937
attack_on_titan,70,164,183,3733
dcu,345,431,398,8493
dr_who,47,110,104,1834
dragonage,218,758,480,6384
haikyuu,96,359,185,4306
hamilton,166,611,330,5116


In [22]:
# Search for groups of interest
import re
from tqdm import tqdm_notebook as tqdm

grouped_patterns = {
    'male': [
        r'(?:\W|\b)he(?:\W|\b)', r'(?:\W|\b)him(?:\W|\b)',
        r'\bmale\b',
        r'boy\b',
        r'^(?!.*(super|wonder|bat)).*man\b',
        r'guy\b', 
        r'\bdad\b', 
        r'husband', 
        r'\bbrother\b', r'\bson\b', 
    ],
    'female': [
        r'(?:\W|\b)she(?:\W|\b)', r'(?:\W|\b)her(?:\W|\b)',
        r'female\b',
        r'^(?!.*(super|wonder|bat)).*\bgirl\b',
        r'^(?!.*(super|wonder|bat)).*\bwoman\b', r'gu+rl', r'\bgii+rl',
        r'\bmom\b', r'mum',
        r'wife', 
        r'\bsister\b',
        r'\blady\b', r'daughter', r'mommy',
    ],
    'trans/nonbinary': [
        r'(?:\W|\b)they(?:\W|\b)', r'(?:\W|\b)them(?:\W|\b)',
        r'(?:\W|\b)xe(?:\W|\b)', r'(?:\W|\b)xem(?:\W|\b)',
        r'it/its',
        r'theythem',
        r'non-binary', r'nonbinary', r'\bnb\b', r'agender', r'neutrois', r'androgynous',
        r'bigender', 
        r'genderfluid', r'gender-fluid',
        r'(?!.*transformers)trans', r'ftm', r'mtf', 
    ],
    'lesbian/gay/bi/pan': [
        r'gay',
        r'lesbian', r'\bhomo',
        r'\bwlw\b', r'\bmlm\b',
        r'bisexual', r'\bbi\b', r'pansexual', r'\bpan\b', 
    ],
    'cis/het': [
        r'\bcis',
        r'\bstraight\b',
        r'\bcishet',
    ]
}
grouped_patterns['lgbtq'] = \
        grouped_patterns['trans/nonbinary'] + \
        grouped_patterns['lesbian/gay/bi/pan'] + \
        [
            r'lgbt',
            r'queer',
            r'\bace\b', r'\basexual', r'aro-ace', r'aro/ace',
        ]
    
outlines = []
demographic_counts = {}
for fandom in fandoms:
    demographic_counts[fandom] = {}
    print(fandom)
    fandom_blog_descriptions = blog_info.loc[blog_info['fandom']==fandom, 'tokenized_blog_description']
    for group_name, patterns in tqdm(grouped_patterns.items()):
        matches = []
        for desc in fandom_blog_descriptions:
            for pattern in patterns:
                if re.search(pattern, desc) is not None:
                    matches.append(desc)
                    break
        group_count = len(matches)
#         if fandom == 'voltron' and group_name == 'bi/pan':
#             pdb.set_trace()
        demographic_counts[fandom][group_name] = group_count
    outlines.append([fandom, *[demographic_counts[fandom][group_name] for group_name in grouped_patterns], len(fandom_blog_descriptions)])
#         print(f'{group_name}\t{group_count}')
#         print()
pd.DataFrame(outlines, columns=['fandom'] + list(grouped_patterns.keys()) + ['total_descriptions']).set_index('fandom')

academia



allmarvel



arthurian



attack_on_titan



dcu


...


tolkien



undertale



voltron



walking_dead


,male,female,trans/nonbinary,lesbian/gay/bi/pan,cis/het,lgbtq,total_descriptions
fandom,,,,,,,
academia,801,1449,783,662,53,1485,8624
allmarvel,3486,5279,2012,2668,314,5039,37002
arthurian,292,409,206,142,22,403,2937
attack_on_titan,428,418,235,187,26,462,3733
dcu,833,804,507,518,88,1039,8493
dr_who,186,261,134,111,18,260,1834
dragonage,565,1209,597,560,94,1222,6384
haikyuu,380,566,232,222,24,594,4306
hamilton,448,892,423,528,47,956,5116


In [2]:
# Find most popular co-occurring terms in blog descriptions
import nltk
import string

stops = nltk.corpus.stopwords.words('english')
stops = [w for w in stops if not w in ['he', 'him', 'she', 'her', 'they', 'them']]
stops += [c for c in string.punctuation]
stops += ['--', "'m", "'s", 'also', "n't", '...', '`', '•', '||', "''", "``", '’', '//', "'re", "'ll"]
stops

# For each fandom, make a counter of terms in blog descriptions
from collections import Counter
fandom_terms = blog_info.groupby('fandom').agg({'tokenized_blog_description': lambda x: ' '.join(x.tolist())})
fandom_words = fandom_terms['tokenized_blog_description'].to_dict()
fandom_word_counters = {}
for fandom, words in fandom_words.items():
    fandom_word_counters[fandom] = Counter([w for w in words.split() if not w in stops])
    
# len(fandom_word_counters)

# from pprint import pprint
# for fandom, ctr in fandom_word_counters.items():
#     print(fandom)
#     pprint(ctr.most_common(30))
#     print()

# Tokenize, lowercase blog descriptions (use NLTK)
Added to preprocess_descriptions.py

In [1]:
# Load fandom blog descriptions
import pandas as pd

fandom_blog_descriptions_path = '/data/tumblr_community_identity/fandom_blog_descriptions_processed.csv'
blog_info = pd.read_csv(fandom_blog_descriptions_path, sep='\t')
# blog_info = pd.read_csv(fandom_blog_descriptions_path, sep='\t', escapechar='\\', error_bad_lines=False)
blog_info

/usr0/home/mamille2/anaconda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tumblelog_id,activity_time_epoch,tumblr_blog_name,tumblr_blog_title,tumblr_blog_description,tumblr_blog_url,is_group_blog,is_primary,is_private,created_time_epoch,updated_time_epoch,timezone,language,blog_classifier,generated_date,tumblog_id,post_count,fandom,fandom_term,processed_tumblr_blog_description
0,123758,1538018782,coldfashion,AP ROCKS YOUR FACE,annelise pruitt i designed the emmy-winning st...,http://coldfashion.tumblr.com/,False,False,False,NaN,NaN,NaN,NaN,safe,20181021,123758,25,star_wars,star wars,annelise pruitt i designed the emmy-winning st...
1,139093,1558154235,hauntedfalcon,Haunted Jaeger,fuck Reylo,https://hauntedfalcon.tumblr.com/,False,False,False,1.196747e+12,1.5463e+12,US/Eastern,en_US,safe,20190518,139093,982,star_wars,reylo,fuck Reylo
2,340785,1546861232,anomalies,party.,"<p>27 years old <br>i like dogs, pokemon, taki...",http://anomalies.tumblr.com/,False,False,False,1.216068e+12,1.54556e+12,US/Pacific,en_US,safe,20190201,340785,2,pokemon,pokemon,"27 years old i like dogs, pokemon, taking pict..."
3,493522,1556894408,hazal,Smitten hedgehogs like tea,"<br>Hazal. <br> 29 years old, vegetarian, web...",https://hazal.tumblr.com/,False,False,False,1.225645e+12,1.54274e+12,Europe/Istanbul,en_US,safe,20190619,493522,10,supernatural,spn,"Hazal. 29 years old, vegetarian, web designer,..."
4,794605,1563544355,sundayriver81,Sundayriver's world,"Italian girl, obsessed with The walking Dead a...",https://sundayriver81.tumblr.com/,False,False,False,1.236731e+12,1.54502e+12,Europe/Amsterdam,en_US,safe,20190719,794605,999,walking_dead,the walking dead,"Italian girl, obsessed with The walking Dead a..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319131,444485579,1541257544,bhluetina,lilyy,<p>potterhead.</p><p>the marauders’</p>,https://bhluetina.tumblr.com/,False,False,False,1.541255e+12,1.54126e+12,US/Eastern,en_US,safe,20181201,444485579,4,harrypotter,potterhead,potterhead. the marauders’
319132,444669851,1541965412,zzellak,Zellak,30/ He/Him/ Them/They. 18+ art. NO MINORS OR ...,https://zzellak.tumblr.com/,False,False,False,1.541347e+12,1.54161e+12,US/Eastern,en_US,adult,20181114,444669851,17,undertale,undertale,30/ He/Him/ Them/They. 18+ art. NO MINORS OR ...
319133,444881744,1567866289,peaches-and-nicotine,♡,"ED TW! || 18 || UK || Not Pro || 5'6"" sw: 218l...",https://peaches-and-nicotine.tumblr.com/,False,False,False,1.541449e+12,1.56527e+12,US/Eastern,en_US,safe,20190908,444881744,88,teen_wolf,tw,"ED TW! || 18 || UK || Not Pro || 5'6"" sw: 218l..."
319134,446026777,1542862673,littlegleekhunter,Pretty Gleek Hunter❤,Pretty Little Liars/Supernatural & Glee❤😍,https://littlegleekhunter.tumblr.com/,False,False,False,1.542070e+12,1.54207e+12,US/Eastern,en_US,safe,20190616,446026777,4,supernatural,supernatural,Pretty Little Liars/Supernatural & Glee❤😍


In [2]:
fandoms = blog_info['fandom'].unique()
print(len(fandoms))
fandoms

42


array(['star_wars', 'pokemon', 'supernatural', 'walking_dead',
       'harrypotter', 'song_ice_fire', 'sherlock', 'tolkien', 'naruto',
       'teen_wolf', 'riverdale', 'dragonage', 'voltron', 'academia',
       'hamilton', 'allmarvel', 'star_trek', 'overwatch',
       'once_upon_a_time', 'haikyuu', 'dr_who', 'undertale', 'homestuck',
       'dcu', 'supercorp', 'arthurian', 'attack_on_titan', 'korra',
       'power rangers', 'stargate', 'ajin', 'world_ends_with_you',
       'youjo senki', 'harry potter', 'marvel', 'hp', 'lord of the rings',
       'reylo', 'rey', 'potterhead', 'klance', 'mcu'], dtype=object)

In [3]:
# Remove error rows with mismatched fandoms
true_fandoms = [
    'ajin',
    'academia',
    'arthurian',
    'dcu',
    'dr_who',
    'dragonage',
    'haikyuu',
    'hamilton',
    'harrypotter',
    'homestuck',
    'allmarvel',
    'naruto',
    'once_upon_a_time',
    'overwatch',
    'pokemon',
    'power rangers',
    'song_ice_fire',
    'riverdale',
    'sherlock',
    'attack_on_titan',
    'star_trek', 
    'star_wars',
    'stargate',
    'supercorp',
    'supernatural',
    'teen_wolf',
    'korra',
    'walking_dead',
    'world_ends_with_you',
    'tolkien',
    'undertale',
    'voltron',
    'youjo senki',
]
print(len(true_fandoms))
blog_info = blog_info[blog_info['fandom'].isin(true_fandoms)]
len(blog_info)

33


319120

In [4]:
# Tokenize blog descriptions (nltk)
import nltk
from tqdm import tqdm_notebook as tqdm

def tokenize_nltk(text):
    return ' '.join([tok.lower() for tok in nltk.word_tokenize(str(text))])

blog_info['tokenized_blog_description'] = [tokenize_nltk(d) for d in tqdm(blog_info['processed_tumblr_blog_description'].tolist())]

/usr0/home/mamille2/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
blog_info.drop_duplicates(subset=['tumblog_id'], inplace=True)
len(blog_info)

/usr0/home/mamille2/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


319116

In [6]:
blog_info.to_csv(fandom_blog_descriptions_path, sep='\t', index=False)

# Old

In [45]:
# Test regular expressions
import re
# pattern = r'^(?!.*trans).*man\b'
# pattern = r'^(?!.*(super|wonder)).*man\b'
pattern = r'^(?!.*transformers)trans'
print(re.search(pattern, 'trans man'))
print(re.search(pattern, 'transman'))
print(re.search(pattern, 'am a transman'))
print(re.search(pattern, 'man'))
print(re.search(pattern, 'am a man'))
print(re.search(pattern, 'superman'))
print(re.search(pattern, 'super man'))
print(re.search(pattern, 'transformers'))

<re.Match object; span=(0, 5), match='trans'>
<re.Match object; span=(0, 5), match='trans'>
None
None
None
None
None
None


In [9]:
# Check for duplicates within fandoms
selected = blog_info[blog_info['fandom']=='power_rangers']
print(len(selected))
print(len(selected.drop_duplicates(subset=['tokenized_blog_description'])))

437
437


In [28]:
# Change blog_info names
blog_info.fandom.unique()

array(['star_wars', 'pokemon', 'supernatural', 'walking_dead',
       'harrypotter', 'song_ice_fire', 'sherlock', 'tolkien', 'naruto',
       'teen_wolf', 'riverdale', 'dragonage', 'voltron', 'academia',
       'hamilton', 'allmarvel', 'star_trek', 'overwatch',
       'once_upon_a_time', 'haikyuu', 'dr_who', 'undertale', 'homestuck',
       'dcu', 'supergirl', 'arthurian', 'attack_on_titan', 'korra',
       'power_rangers', 'stargate', 'ajin', 'world_ends_with_you',
       'youjo senki'], dtype=object)

In [27]:
blog_info['fandom'] = blog_info['fandom'].replace({'supercorp': 'supergirl', 'power rangers': 'power_rangers'})

In [29]:
blog_info.to_csv(fandom_blog_descriptions_path, sep='\t', index=False)

In [53]:
# Tokenize blog descriptions (spaCy)
import spacy

nlp = spacy.load('en', disable=['tagger', 'parser', 'ner'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))
def tokenize_spacy(text):
    full_text = ''
    full_text = ''
    if len(str(text)) > 100000:
        print('Description too long')
        return full_text
    doc = nlp(str(text))
    for sent in doc.sents:
        full_text += ' '.join([tok.text.lower() for tok in sent]) + ' '
    return full_text.strip()

In [51]:
# Tokenize blog descriptions (stanza)
import stanza

nlp = stanza.Pipeline(lang='en', processors='tokenize')
# done_texts = []
def tokenize_stanza(text):
    full_text = ''
    doc = nlp(str(text))
    for sent in doc.sentences:
        full_text += ' '.join([tok.text for tok in sent.tokens]) + ' '
    return full_text.strip()

2020-11-12 11:40:03 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |

2020-11-12 11:40:03 INFO: Use device: cpu
2020-11-12 11:40:03 INFO: Loading: tokenize
2020-11-12 11:40:03 INFO: Done loading processors!
